In [1]:
!pip install git+https://github.com/Seokhyeon-Park/hanspell.git

  Cloning https://github.com/Seokhyeon-Park/hanspell.git to /tmp/pip-req-build-n_negujv
  Running command git clone --filter=blob:none --quiet https://github.com/Seokhyeon-Park/hanspell.git /tmp/pip-req-build-n_negujv
  Resolved https://github.com/Seokhyeon-Park/hanspell.git to commit 87c740ec6ad6f8c418fcac653d0e5d8ae5b7c1d9
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [1]:
import os
import inspect
from hanspell import spell_checker

# spell_checker.py 파일 경로 찾기
spell_checker_file = inspect.getfile(spell_checker)

# 파일 내용 출력
try:
    with open(spell_checker_file, "r", encoding="utf-8") as file:
        content = file.read()
        print("=== spell_checker.py 내용 ===")
        print(content)
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {spell_checker_file}")
except Exception as e:
    print(f"오류 발생: {e}")


=== spell_checker.py 내용 ===
# -*- coding: utf-8 -*-
"""
Python용 한글 맞춤법 검사 모듈
"""

import requests
import json
import time
import sys
import re
from collections import OrderedDict
import xml.etree.ElementTree as ET

from . import __version__
from .response import Checked
from .constants import base_url
from .constants import CheckResult

_agent = requests.Session()
PY3 = sys.version_info[0] == 3

# print("check edit spell checker!!!")

def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>','')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):
    """
    매개변수로 입력받은 한글 문장의 맞춤법을 체크합니다.
    """
    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    # 최대 500자까지 가능.
    if len(text) > 500:
        return Checked(result=False)

    payload = {
        

In [2]:
import re
import inspect
from hanspell import spell_checker

# spell_checker.py 파일 경로 찾기
spell_checker_file = inspect.getfile(spell_checker)

# 파일 읽기
try:
    with open(spell_checker_file, "r", encoding="utf-8") as file:
        content = file.read()

    # 필요한 값 수정
    content = re.sub(
        r"'passportKey':\s*'[^']+'",
        "'passportKey': 'd2cd9d51cafed2595fe629dc5f80d2fcf0b89529'",
        content
    )
    content = re.sub(
        r"'_':\s*\d+",
        "'_': 1737107968111",
        content
    )

    # 수정된 내용을 같은 이름으로 저장
    with open(spell_checker_file, "w", encoding="utf-8") as file:
        file.write(content)

    print(f"수정된 내용이 {spell_checker_file}에 저장되었습니다.")

except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {spell_checker_file}")
except Exception as e:
    print(f"오류 발생: {e}")


수정된 내용이 /usr/local/lib/python3.9/dist-packages/hanspell/spell_checker.py에 저장되었습니다.


In [1]:
from hanspell import spell_checker

# 검사할 문장
text = "안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다."

# 맞춤법 검사
result = spell_checker.check(text)

# 결과 출력
print("원문:", result.original)
print("교정된 문장:", result.checked)
print("오류 수:", result.errors)


원문: 안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.
교정된 문장: 안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.
오류 수: 4


In [2]:
import pandas as pd
import os
from hanspell import spell_checker

# 경로 설정
train_dir = "./Data/None"
output_dir = "./Data/SC"
os.makedirs(output_dir, exist_ok=True)  # SC 폴더 생성

# 파일 처리
for train_file in sorted(os.listdir(train_dir)):
    if train_file.endswith(".csv"):
        # 파일 읽기
        file_path = os.path.join(train_dir, train_file)
        data = pd.read_csv(file_path)

        # 맞춤법 검사 및 수정
        new_rows = []
        for idx, row in data.iterrows():
            try:
                result = spell_checker.check(row["comments"])
                corrected_comment = result.checked  # 교정된 문장
            except Exception as e:
                print(f"Error processing comment: {row['comments']}. Error: {e}")
                corrected_comment = row["comments"]  # 오류 시 원문 유지
            
            # 기존 행 그대로 추가
            new_rows.append(row)

            # 교정된 문장을 새로운 행으로 추가
            corrected_row = row.copy()
            corrected_row["comments"] = corrected_comment
            corrected_row["hate"] = row["hate"]  # 원래 hate 값 유지
            new_rows.append(corrected_row)

        # 새로운 데이터프레임 생성
        updated_data = pd.DataFrame(new_rows)

        # 중복 제거 (comments 열 기준)
        updated_data = updated_data.drop_duplicates(subset=["comments"], keep="first")

        # 출력 파일 저장 (UTF-8 with BOM)
        output_file = os.path.join(output_dir, f"SC_{train_file.split('_')[1]}")
        updated_data.to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"Processed and deduplicated file saved to {output_file}")


Processed and deduplicated file saved to ./Data/SC/SC_1.csv
Processed and deduplicated file saved to ./Data/SC/SC_2.csv
Processed and deduplicated file saved to ./Data/SC/SC_3.csv
Processed and deduplicated file saved to ./Data/SC/SC_4.csv
Processed and deduplicated file saved to ./Data/SC/SC_5.csv


In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import f1_score
import os

# 사용자 정의 데이터셋 클래스
class SimpleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# 경로 설정
train_dir = "./Data/SC"  # None을 SC로 변경
test_file = "./Data/Test.csv"
summary_file = "./Data/Summary.csv"

# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3개 클래스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 학습 및 평가 결과 저장
results = []

# 테스트 데이터 준비
test_data = pd.read_csv(test_file)
test_texts = test_data["comments"].tolist()
test_labels = test_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()
test_dataset = SimpleDataset(test_texts, test_labels, tokenizer, max_len=128)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 루프
for train_file in sorted(os.listdir(train_dir)):
    if train_file.endswith(".csv"):
        train_data = pd.read_csv(os.path.join(train_dir, train_file))
        train_texts = train_data["comments"].tolist()
        train_labels = train_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()

        train_dataset = SimpleDataset(train_texts, train_labels, tokenizer, max_len=128)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        # 옵티마이저 설정
        optimizer = AdamW(model.parameters(), lr=5e-5)

        # 학습
        model.train()
        for epoch in range(3):  # 3 에포크
            total_loss = 0
            for batch in train_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"{train_file} - Epoch {epoch + 1} - Loss: {total_loss:.4f}")

        # 평가
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, axis=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        f1_micro = f1_score(all_labels, all_preds, average="micro")
        f1_macro = f1_score(all_labels, all_preds, average="macro")
        print(f"Test Results for {train_file}: F1 Micro = {f1_micro:.4f}, F1 Macro = {f1_macro:.4f}")
        results.append({"File": train_file, "F1 Micro": f1_micro, "F1 Macro": f1_macro})

# 결과 저장 또는 추가
if os.path.exists(summary_file):
    # 기존 파일이 있는 경우: 기존 데이터를 읽어서 새로운 데이터를 추가
    existing_results = pd.read_csv(summary_file)
    results_df = pd.concat([existing_results, pd.DataFrame(results)], ignore_index=True)
else:
    # 기존 파일이 없는 경우: 새로운 데이터만 저장
    results_df = pd.DataFrame(results)

results_df.to_csv(summary_file, index=False)
print(f"Summary updated and saved to {summary_file}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SC_1.csv - Epoch 1 - Loss: 31.6140
SC_1.csv - Epoch 2 - Loss: 31.5807
SC_1.csv - Epoch 3 - Loss: 30.8945
Test Results for SC_1.csv: F1 Micro = 0.3631, F1 Macro = 0.2604


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SC_2.csv - Epoch 1 - Loss: 30.3849
SC_2.csv - Epoch 2 - Loss: 29.5372
SC_2.csv - Epoch 3 - Loss: 27.8773
Test Results for SC_2.csv: F1 Micro = 0.3800, F1 Macro = 0.3526


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SC_3.csv - Epoch 1 - Loss: 30.5410
SC_3.csv - Epoch 2 - Loss: 28.7139
SC_3.csv - Epoch 3 - Loss: 28.1440
Test Results for SC_3.csv: F1 Micro = 0.3992, F1 Macro = 0.3607


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SC_4.csv - Epoch 1 - Loss: 31.0362
SC_4.csv - Epoch 2 - Loss: 29.7299
SC_4.csv - Epoch 3 - Loss: 28.3517
Test Results for SC_4.csv: F1 Micro = 0.4013, F1 Macro = 0.3877


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


SC_5.csv - Epoch 1 - Loss: 30.5284
SC_5.csv - Epoch 2 - Loss: 29.2253
SC_5.csv - Epoch 3 - Loss: 28.2508
Test Results for SC_5.csv: F1 Micro = 0.4013, F1 Macro = 0.3568
Summary updated and saved to ./Data/Summary.csv
